In [175]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go

In [176]:
np.random.seed(0)
X = np.random.uniform(-10, 10, 100)
epsilon = np.random.normal(0, 0.1, 100)
y = 3 * X + 4 + epsilon

In [177]:
# Convert the data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).view(-1, 1)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1) 

In [178]:
# Define the linear regression model
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)  # One feature to one output

    def forward(self, x):
        return self.linear(x)
    
model_standard_SGD = LinearRegressionModel()
model_custom_SGD = LinearRegressionModel()

In [179]:
def custom_gradient_descent(model, X_tensor, y_tensor, learning_rate=0.01, num_epochs=100):
    criterion  = torch.nn.MSELoss()
    loss_values = []

    for _ in range(num_epochs):
        y_pred = model(X_tensor)
        loss = criterion(y_pred, y_tensor)
        loss.backward()
        loss_values.append(loss.item())

        # Update weights and bias manually
        with torch.no_grad():
            model.linear.weight -= learning_rate * model.linear.weight.grad
            model.linear.bias -= learning_rate * model.linear.bias.grad

            # Manually zero gradients after updating
            model.linear.weight.grad.zero_()
            model.linear.bias.grad.zero_()

    return loss_values

In [180]:
# Define the gradient descent function
def gradient_descent(model, X_tensor, y_tensor, learning_rate=0.01, num_epochs=100):
    criterion = nn.MSELoss()  # Mean Squared Error
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)  # Stochastic Gradient Descent
    
    loss_values = []

    for _ in range(num_epochs):
        y_pred = model(X_tensor)
        loss = criterion(y_pred, y_tensor)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_values.append(loss.item())

    return loss_values

In [181]:
loss_standard = gradient_descent(model_standard_SGD, X_tensor, y_tensor)
loss_custom = custom_gradient_descent(model_custom_SGD, X_tensor, y_tensor)

In [182]:
# Visualize the training loss for both models using Plotly
fig = go.Figure()

# Add traces for both models
fig.add_trace(go.Scatter(x=np.arange(len(loss_standard)), y=loss_standard, mode='lines', name='Standard SGD'))
fig.add_trace(go.Scatter(x=np.arange(len(loss_custom)), y=loss_custom, mode='lines', name='Custom SGD'))

# Customize the layout
fig.update_layout(
    title="Training Loss",
    xaxis=dict(title="Epochs"),
    yaxis=dict(title="Loss (MSE)"),
    showlegend=True
)

# Show the plot
fig.show()

In [183]:
# Plot the results using Plotly
predicted_standard = model_standard_SGD(X_tensor).detach().numpy()
predicted_custom = model_custom_SGD(X_tensor).detach().numpy()

scatter = go.Scatter(x=X, y=y, mode='markers', name='True data')
line_standard = go.Scatter(x=X, y=predicted_standard.flatten(), mode='lines', name='Fitted line standard SGD', line=dict(color='red'))
line_custom = go.Scatter(x=X, y=predicted_custom.flatten(), mode='lines', name='Fitted line custom SGD', line=dict(color='green'))

layout = go.Layout(
    title="Linear Regression Fit",
    xaxis=dict(title="X"),
    yaxis=dict(title="y"),
    showlegend=True
)

fig = go.Figure(data=[scatter, line_standard, line_custom], layout=layout)
fig.show()

print(f"Learned parameters (standard): W = {model_standard_SGD.linear.weight.item():.4f}, b = {model_standard_SGD.linear.bias.item():.4f}")
print(f"Learned parameters (custom): W = {model_custom_SGD.linear.weight.item():.4f}, b = {model_custom_SGD.linear.bias.item():.4f}")

Learned parameters (standard): W = 2.9919, b = 3.5535
Learned parameters (custom): W = 2.9904, b = 3.4616
